In [ ]:
import os
import json
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from src.data import data_layer
from src.model.model_utils import calculateStatistics
from src.model.model_GAN import FeedForwardModelWithNA_GAN_Ensembled
from src.model.model_RtnFcst import FeedForwardModelWithNA_Return_Ensembled
from src.utils import deco_print
from src.utils import load_dataframe
from src.utils import sharpe

##### Model Configurations

In [ ]:
task_id = 1
logdirs = ['output/Task-%d-Trial-%d/sharpe' %(task_id, k) for k in range(9)]
with open('config/config.json') as file:
    config = json.load(file)
    
logdirs_RF = ['output_RF/Task-%d-Trial-%d' %(task_id, k) for k in range(9)]
with open('config_RF/config_RF_%d.json'%(task_id)) as file:
    config_RF = json.load(file)

##### Load Data

In [3]:
dl = data_layer.DataInRamInputLayer(config['individual_feature_file'],
                                    pathMacroFeature=config['macro_feature_file'])
meanMacroFeature, stdMacroFeature = dl.getMacroFeatureMeanStd()
dl_valid = data_layer.DataInRamInputLayer(config['individual_feature_file_valid'],
                                          pathMacroFeature=config['macro_feature_file_valid'],
                                          meanMacroFeature=meanMacroFeature,
                                          stdMacroFeature=stdMacroFeature)
dl_test = data_layer.DataInRamInputLayer(config['individual_feature_file_test'],
                                         pathMacroFeature=config['macro_feature_file_test'],
                                         meanMacroFeature=meanMacroFeature,
                                         stdMacroFeature=stdMacroFeature)

##### Load Model

In [4]:
tf.reset_default_graph()
global_step = tf.train.get_or_create_global_step()
model = FeedForwardModelWithNA_GAN_Ensembled(logdirs, config, 'test', config['tSize'], global_step=global_step)
model_valid = FeedForwardModelWithNA_GAN_Ensembled(logdirs, config, 'test', config['tSize_valid'], force_var_reuse=True, global_step=global_step)
model_test = FeedForwardModelWithNA_GAN_Ensembled(logdirs, config, 'test', config['tSize_test'], force_var_reuse=True, global_step=global_step)
model_plot = FeedForwardModelWithNA_GAN_Ensembled(logdirs, config, 'plot', 1, force_var_reuse=True, global_step=global_step)
gpu_options = tf.GPUOptions(allow_growth=True)
sess_config = tf.ConfigProto(gpu_options=gpu_options)
sess = tf.Session(config=sess_config)

In [ ]:
INITIAL_train = model.getZeroInitialState()
INITIAL_valid = model.getNextInitialState(sess, dl, initial_state=INITIAL_train)
INITIAL_test = model_valid.getNextInitialState(sess, dl_valid, initial_state=INITIAL_valid)

##### Model Performance

In [ ]:
w = model.getWeightWithData(sess, dl, initial_state=INITIAL_train, normalized=True)
w_valid = model_valid.getWeightWithData(sess, dl_valid, initial_state=INITIAL_valid, normalized=True)
w_test = model_test.getWeightWithData(sess, dl_test, initial_state=INITIAL_test, normalized=True)

In [ ]:
Ftrain = model.getNormalizedSDFFactor(sess, dl, initial_state=INITIAL_train)
Fvalid = model_valid.getNormalizedSDFFactor(sess, dl_valid, initial_state=INITIAL_valid)
Ftest = model_test.getNormalizedSDFFactor(sess, dl_test, initial_state=INITIAL_test)
sdf_norm_ensemble = np.concatenate([Ftrain, Fvalid, Ftest])
np.save(os.path.join(logdir, 'sdf_normalized_ensemble.npy'), sdf_norm_ensemble)

In [8]:
SR_train = sharpe(Ftrain)
SR_valid = sharpe(Fvalid)
SR_test = sharpe(Ftest)
deco_print('SDF Portfolio Sharpe Ratio: Train %0.2f\tValid %0.2f\tTest %0.2f' %(SR_train, SR_valid, SR_test))

>==================> SDF Portfolio Sharpe Ratio: Train 3.02	Valid 1.39	Test 0.77


##### Predictive Performance

In [9]:
tf.reset_default_graph()
global_step = tf.train.get_or_create_global_step()
model_plot_RF = FeedForwardModelWithNA_Return_Ensembled(logdirs_RF, config_RF, 'plot', global_step=global_step)
gpu_options = tf.GPUOptions(allow_growth=True)
sess_config = tf.ConfigProto(gpu_options=gpu_options)
sess = tf.Session(config=sess_config)

In [10]:
dl_RF_train = data_layer.DataInRamInputLayer(config['individual_feature_file'])
dl_RF_valid = data_layer.DataInRamInputLayer(config['individual_feature_file_valid'])
dl_RF_test = data_layer.DataInRamInputLayer(config['individual_feature_file_test'])

In [ ]:
beta_train = model_plot_RF.getPrediction(sess, dl_RF_train)
beta_valid = model_plot_RF.getPrediction(sess, dl_RF_valid)
beta_test = model_plot_RF.getPrediction(sess, dl_RF_test)

In [12]:
# EV, XS-R2
EV_train, XSR2_train, WXSR2_train = calculateStatistics(beta_train, dl)
EV_valid, XSR2_valid, WXSR2_valid = calculateStatistics(beta_valid, dl_valid)
EV_test, XSR2_test, WXSR2_test = calculateStatistics(beta_test, dl_test)

In [13]:
deco_print('Explained Variation: Train %0.2f\tValid %0.2f\tTest %0.2f' %(EV_train, EV_valid, EV_test))
deco_print('XS-R2: Train %0.2f\tValid %0.2f\tTest %0.2f' %(XSR2_train, XSR2_valid, XSR2_test))
deco_print('(Weighted) XS-R2: Train %0.2f\tValid %0.2f\tTest %0.2f' %(WXSR2_train, WXSR2_valid, WXSR2_test))

>==================> Explained Variation: Train 0.18	Valid 0.08	Test 0.07
>==================> XS-R2: Train -0.02	Valid 0.04	Test 0.04
>==================> (Weighted) XS-R2: Train 0.12	Valid 0.00	Test 0.22
